In [5]:
import re

file_path = 'Book\Structuralisme_2_Dosse.txt'

# Read the input file
with open(file_path, 'r', encoding='utf-8') as f:
    text = f.read()

# Remove isolated footnote markers like [1], [2]
text = re.sub(r"\[\d+\] ?↑?.*?(\n|$)", "", text)

# Remove footnotes of the format "1. Some text" at the end of each chapter
text = re.sub(r"(\n|^)\d+\.\s+.+?(?=\n\n|\Z)", "", text, flags=re.DOTALL)

# Write the cleaned text back to a file
with open(file_path, 'w', encoding='utf-8') as f:
    f.write(text)
